# Head Pose Estimation Prediction Alignment

<a target="_blank" href="https://colab.research.google.com/github/pcr-upm/opal23_headpose/blob/main/notebooks/align_predictions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook shows how to use our alignment code to remove systematic errors from network predictions on cross-dataset benchmarks.

Download evaluation code

In [1]:
!wget -nc https://raw.githubusercontent.com/pcr-upm/opal23_headpose/main/test/evaluator.py

--2024-01-19 12:33:38--  https://raw.githubusercontent.com/pcr-upm/opal23_headpose/main/test/evaluator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4565 (4.5K) [text/plain]
Saving to: ‘evaluator.py’

evaluator.py        100%[===================>]   4.46K  --.-KB/s    in 0s      

2024-01-19 12:33:39 (51.9 MB/s) - ‘evaluator.py’ saved [4565/4565]



In [2]:
import numpy as np
from scipy.spatial.transform import Rotation

from evaluator import Evaluator

np.random.seed(0)

Create some random annotations and simulate network prediction errors and systematic alignment errors

In [3]:
# Create sample annotations in the range [-60, 60]
ann_euler = np.random.rand(1000, 3) * 120 - 60

# Assume network predictions add some gaussian noise
pred_euler = ann_euler + np.random.normal(loc=0.0, scale=3.0, size=ann_euler.shape)

# Add systematic noise to simulate cross-dataset alignment errors
alignment_error = np.random.rand(3) * 10 - 5
pred_euler += alignment_error

print('Sample annotation', ann_euler[0])
print('Sample prediction', pred_euler[0])
print('Alignment error', alignment_error)

Sample annotation [ 5.85762047 25.82272396 12.33160513]
Sample prediction [12.35355223 24.8916     14.69210784]
Alignment error [1.09454347 2.65418735 0.81098038]


In [4]:
# Compute metrics
ann_matrix= Rotation.from_euler('XYZ', ann_euler, degrees=True).as_matrix()
pred_matrix = Rotation.from_euler('XYZ', pred_euler, degrees=True).as_matrix()

evaluator = Evaluator(ann_matrix, pred_matrix)
print('Unaligned results:')
print('MAE', np.mean(evaluator.compute_mae()))
print('GE', np.mean(evaluator.compute_ge()))

Unaligned results:
MAE 2.742214109828782
GE 5.41753466733105


Prediction alignment can be done with a single line of code

In [5]:
evaluator.align_predictions()
print('Aligned results:')
print('MAE', np.mean(evaluator.compute_mae()))
print('GE', np.mean(evaluator.compute_ge()))

Aligned results:
MAE 2.5736892090096606
GE 4.931510920290861
